# Test if a Binary Tree Satisfies the BST Property

**Write a program that takes as input a binary tree and checks if the tree satisfies
the BST property.**

## Solution

A direct approach, based on the definition of a BST, is to begin with the root, and 
compute the maximum key stored in the root's left subtree, and the minimum key in 
the root's right subtree.  We check that the key at the root is greater than or 
equal to the maximum from the left subtree and less than or equal to the minimum
from the right subtree.  If both these checks pass, we recursively check the root's 
left and right subtree.  If either check fails, we return false.

Computing the minimum key in a binary tree is straightforward: we take the minimum
of the key stored at its root, the minimum key of the left subtree, and the minimum 
key of the right subtree.  The maximum key is computed similarly.  Note that the 
minimum can be in either subtree, since a general binary tree may not satisfy the 
BST property.

The problem with this approach is that it will repeatedly traverse subtrees.  In the
worst-case, when the tree is a BST and each node's left child is empty, the 
complexity is $O(n^2)$, where $n$ is the number of nodes.  The complexity can be 
improved to $O(n)$ by caching the largest and smallest keys at each node; this 
requires $O(n)$ additional storage for the cache.

We now present two approaches which have $O(n)$ time complexity.

The first approach is to check constraints on the values for each subtree.  The 
initial constraint comes from the root.  Every node in its left (right) subtree
must have a key less than or equal (greater than or equal) to the key at the root.
This idea generalizes: if all nodes in a tree must have keys in the range $[l,u]$,
and the key at the root is $w$ (which itself must be between $[l,u]$, otherwise the
requirement is violated at the root itself), then all keys in the left subtree must 
be in the range $[l,w]$, and all keys stored in the right subtree must be in the
range $[w,u]$.

In [23]:
class BinaryTreeNode:
    
    def __init__(self, data=None, left=None, right=None):
        self.data = data
        self.left = left
        self.right = right
        
def is_binary_tree_bst(tree, low_range=float('-inf'), high_range=float('inf')):
    if not tree:
        return True
    elif not low_range <= tree.data <= high_range:
        return False
    return (is_binary_tree_bst(tree.left, low_range, tree.data)
            and is_binary_tree_bst(tree.right, tree.data, high_range))

example_tree = BinaryTreeNode(data=1)
example_tree.right = BinaryTreeNode(data=2)
example_tree.left = BinaryTreeNode(data=3)
example_tree.right.right = BinaryTreeNode(data=4)
example_tree.right.left = BinaryTreeNode(data=5)
example_tree.left.left = BinaryTreeNode(data=6)
example_tree.left.left.right = BinaryTreeNode(data=7)
example_tree.left.left.left = BinaryTreeNode(data=8)

print("example non BST bintree is a BST?: {0}".format(is_binary_tree_bst(example_tree)))

example_tree = BinaryTreeNode(data=19)
example_tree.left = BinaryTreeNode(data=7)
example_tree.left.left = BinaryTreeNode(data=3)
example_tree.left.left.left = BinaryTreeNode(data=2)
example_tree.left.left.right = BinaryTreeNode(data=5)
example_tree.right = BinaryTreeNode(data=43)
example_tree.right.left = BinaryTreeNode(data=23)
example_tree.right.left.right = BinaryTreeNode(data=37)
example_tree.right.left.right.left = BinaryTreeNode(data=29)
example_tree.right.left.right.left.right = BinaryTreeNode(data=31)
print("example BST is a BST?: {0}".format(is_binary_tree_bst(example_tree)))

example non BST bintree is a BST?: False
example BST is a BST?: True


The time complexity is $O(n)$ and the additional space complexity is $O(h)$, where
$h$ is the height of the tree.

Alternatively, we can use the fact that an inorder traversal visits keys in sorted order.  
Furthermore, if an inorder traversal of a binary tree visits keys in sorted order, then 
that binary tree must be a BST.  (This follows directly from the definition of a BST
and the definition of an inorder walk.)  Thus we can check the BST property by 
performing an inorder traversal, recording the key stored at the last visited node.
Each time a new node is visited, its key is compared with the key of the previously
visited node.  If at any step in the walk, the key at the previously visited node is
greater than the node currently being visited, we have a violation of the BST 
property.

All these approaches explore the left subtree first.  Therefore, even if the BST
property does not hold at a node which is close to the root (eg the key stored at 
the right child is less than the key stored at the root), their time complexity is
still $O(n)$.

We can search for violations of the BST property in a BFS manner, thereby reducing
the time complexity when the property is violated at a node whose depth is small.

Specifically, we use a queue, where each queue entry contains a node, as well as 
an upper and a lower bound on the keys stored at the subtree rooted at that node.
The queue is intiialized to the root, with lower bound $-\infty$  and upper bound
$\infty$.  We iteratively check the constraint on each node.  If it violates the
constraint we stop - the BST property has been violated.  Otherwise, we add its
children along with the corresponding constraint.

IF the BST is property is violated in a subtree consisting of nodes within a 
particular depth, the violation will be discovered without visiting any modes at
a greater depth.  This is because each time we enqeueue an entry, the lower and 
upper bounds on the node's key are the tightest posible.

In [24]:
import collections

def is_binary_tree_bst(tree: BinaryTreeNode) -> bool:
    QueueEntry = collections.namedtuple('QueueEntry',
                                        ('node', 'lower', 'upper'))
    
    bfs_queue = collections.deque(
        [QueueEntry(tree, float('-inf'), float('inf'))])
    
    while bfs_queue:
        front = bfs_queue.popleft()
        if front.node:
            if not front.lower <= front.node.data <= front.upper:
                return False
            bfs_queue.extend(
                (QueueEntry(front.node.left, front.lower, front.node.data),
                 QueueEntry(front.node.right, front.node.data, front.upper)))
    return True

print("example BST is a BST?: {0}".format(is_binary_tree_bst(example_tree)))

example BST is a BST?: True


The time complexity is $O(n)$, and the additional space complexity is $O(n)$.

[References](../reference/14.1.md)